# Choose X random windows of size Y in a genome that overlap gene models

Based on an input annotation file (e.g. gff3), choose 1000 random windows that contain genes

First, read in the input file. This one is the Sorghum bicolor gff3 from Phytozome v12, available at https://genome.jgi.doe.gov/portal/pages/dynamicOrganismDownload.jsf?organism=Phytozome:

In [74]:
import numpy as np

#read in important columns into dataframe
Chr = np.loadtxt( "/Users/emilywork/Downloads/Sbicolor_313_v3.1.gene.gff3", usecols=[0], dtype="str", unpack=True )
Ann = np.loadtxt( "/Users/emilywork/Downloads/Sbicolor_313_v3.1.gene.gff3", usecols=[2], dtype="str", unpack=True )
Start = np.loadtxt( "/Users/emilywork/Downloads/Sbicolor_313_v3.1.gene.gff3", usecols=[3], dtype="int", unpack=True )
Stop = np.loadtxt( "/Users/emilywork/Downloads/Sbicolor_313_v3.1.gene.gff3", usecols=[4], dtype="int", unpack=True )

gffdata = np.column_stack([Chr,Ann,Start,Stop])

##np array
print("Data type:", type(gffdata))
print(gffdata[0:5])
print("\nNumber of rows:", len(gffdata),"\n")

##pandas df
import pandas as pd
df = pd.DataFrame(data=gffdata, columns=['Chr', 'Ann', 'Start','Stop'])
print("Data type:", type(df),"\n",df[0:5])

Data type: <class 'numpy.ndarray'>
[['Chr01' 'gene' '1951' '2616']
 ['Chr01' 'mRNA' '1951' '2616']
 ['Chr01' 'CDS' '1951' '2454']
 ['Chr01' 'CDS' '2473' '2616']
 ['Chr01' 'gene' '11180' '14899']]

Number of rows: 426522 

Data type: <class 'pandas.core.frame.DataFrame'> 
      Chr   Ann  Start   Stop
0  Chr01  gene   1951   2616
1  Chr01  mRNA   1951   2616
2  Chr01   CDS   1951   2454
3  Chr01   CDS   2473   2616
4  Chr01  gene  11180  14899


Then, get just the entries that are annotated as being a gene:

In [121]:
# select rows containing 'gene'
genes = df.loc[df['Ann'] =="gene"]
print("Number of rows:", len(genes),"\n",genes[0:5])

Number of rows: 34211 
        Chr   Ann  Start   Stop
0    Chr01  gene   1951   2616
4    Chr01  gene  11180  14899
19   Chr01  gene  23399  24152
24   Chr01  gene  22391  42443
165  Chr01  gene  52891  53594


Get random 5 kb windows that contain at least part of a gene

In [167]:
randos = genes.sample(1000)
print(randos[0:5])

          Chr   Ann     Start      Stop  Length
341124  Chr08  gene  19265296  19268301    3005
290955  Chr06  gene  55347385  55349551    2166
239893  Chr05  gene   1574101   1577978    3877
251492  Chr05  gene  33183891  33192725    8834
170509  Chr03  gene  63364389  63369675    5286


In [173]:
import math

randos['WinStart'] = 0
for i in randos.index:
    randos.loc[i,'WinStart'] = math.floor(pd.to_numeric(randos.loc[i,'Start'])/1000)*1000
    randos.loc[i,'WinEnd'] = randos.loc[i,'WinStart']+5000
    
print(randos[0:5])

          Chr   Ann     Start      Stop  Length  WinStart      WinEnd
341124  Chr08  gene  19265296  19268301    3005  19265000  19270000.0
290955  Chr06  gene  55347385  55349551    2166  55347000  55352000.0
239893  Chr05  gene   1574101   1577978    3877   1574000   1579000.0
251492  Chr05  gene  33183891  33192725    8834  33183000  33188000.0
170509  Chr03  gene  63364389  63369675    5286  63364000  63369000.0


In [179]:
randos.to_csv('/Users/emilywork/Desktop/RandomWin.txt', header=None, index=None, sep='\t', mode='a')